In [1]:
!pip list|grep ibm-watson-studio-lib 

ibm-watson-studio-lib       3.0.6


In [2]:
!pip list|grep boto

boto3                       1.17.46
botocore                    1.20.88


In [3]:
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()


In [4]:
#接続オブジェクトのリスト
assets=wslib.list_connections()
wslib.show(assets)

# 0
{'name': '200322icosbucket',
 'description': '',
 'asset_id': '5bdf8401-22de-4425-a421-8cff89cce8cf',
 'asset_type': 'connection',
 'tags': []}
# 1
{'name': 'DB2WHOC_HDM',
 'description': '',
 'asset_id': 'd26a6804-b8b0-41e8-8078-a725ff6e6f32',
 'asset_type': 'connection',
 'tags': []}


In [5]:
#接続オブジェクトの情報を取得
connection_config=wslib.get_connection('200322icosbucket')
#connection_config

In [6]:
#ICOS接続
from botocore.client import Config
import ibm_boto3


cosclient = ibm_boto3.client("s3",
    ibm_api_key_id=connection_config['api_key'],
    ibm_service_instance_id=connection_config['resource_instance_id'],
    ibm_auth_endpoint=connection_config['iam_url'],
    config=Config(signature_version="oauth"),
    endpoint_url='https://'+connection_config['url']
)

In [7]:
#バケットのオブジェクトリストを取得
listobject=cosclient.list_objects(Bucket=connection_config['bucket'])
listobject['Contents']

[{'Key': 'Cond4n_e.csv',
  'LastModified': datetime.datetime(2020, 4, 22, 6, 9, 45, 212000, tzinfo=tzlocal()),
  'ETag': '"a2f6827a0f9433a1d91520607747276b"',
  'Size': 134617,
  'StorageClass': 'STANDARD',
  'Owner': {'DisplayName': '52aa9e0c-0aa7-45e5-b231-1a65d1a57a1a',
   'ID': '52aa9e0c-0aa7-45e5-b231-1a65d1a57a1a'}},
 {'Key': 'Cond4n_e104.csv',
  'LastModified': datetime.datetime(2020, 4, 22, 6, 9, 43, 413000, tzinfo=tzlocal()),
  'ETag': '"fdb3803425be82d35c6ed178a14c4305"',
  'Size': 146,
  'StorageClass': 'STANDARD',
  'Owner': {'DisplayName': '52aa9e0c-0aa7-45e5-b231-1a65d1a57a1a',
   'ID': '52aa9e0c-0aa7-45e5-b231-1a65d1a57a1a'}},
 {'Key': 'Cond4n_e104.parquet',
  'LastModified': datetime.datetime(2021, 9, 30, 8, 7, 8, 655000, tzinfo=tzlocal()),
  'ETag': '"09882233f58af6073d0516933749873c"',
  'Size': 4208,
  'StorageClass': 'STANDARD',
  'Owner': {'DisplayName': '52aa9e0c-0aa7-45e5-b231-1a65d1a57a1a',
   'ID': '52aa9e0c-0aa7-45e5-b231-1a65d1a57a1a'}},
 {'Key': 'output/mf_C

In [8]:
#csvのダウンロード
import pandas as pd
from io import StringIO
body=cosclient.get_object(Bucket=connection_config['bucket'],Key='Cond4n_e104.csv')['Body']
df=pd.read_csv(StringIO(body.read().decode('utf-8')))
df.to_csv('Cond4new.csv', index=False)

In [9]:
!ls

Cond4new.csv


In [10]:
!cat Cond4new.csv


M_CD,UP_TIME,POWER,TEMP,ERR_CD
104,34,1003,244,0
104,35,1003,244,0
104,36,1003,244,0
104,37,1003,244,0
104,38,973,244,0


In [11]:
#csvのアップロード
cosclient.put_object(Bucket=connection_config['bucket'],
                     Key='Cond4new.csv', Body='Cond4new.csv')

{'ResponseMetadata': {'RequestId': '2938b57b-da3f-4474-a3e1-8a290e21b371',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 24 Jan 2022 06:11:57 GMT',
   'x-clv-request-id': '2938b57b-da3f-4474-a3e1-8a290e21b371',
   'server': 'Cleversafe',
   'x-clv-s3-version': '2.5',
   'x-amz-request-id': '2938b57b-da3f-4474-a3e1-8a290e21b371',
   'etag': '"e648c1551e0c67f2b4494dfed0e9174c"',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"e648c1551e0c67f2b4494dfed0e9174c"'}

In [12]:
listobject=cosclient.list_objects(Bucket=connection_config['bucket'])
listobject['Contents']

[{'Key': 'Cond4n_e.csv',
  'LastModified': datetime.datetime(2020, 4, 22, 6, 9, 45, 212000, tzinfo=tzlocal()),
  'ETag': '"a2f6827a0f9433a1d91520607747276b"',
  'Size': 134617,
  'StorageClass': 'STANDARD',
  'Owner': {'DisplayName': '52aa9e0c-0aa7-45e5-b231-1a65d1a57a1a',
   'ID': '52aa9e0c-0aa7-45e5-b231-1a65d1a57a1a'}},
 {'Key': 'Cond4n_e104.csv',
  'LastModified': datetime.datetime(2020, 4, 22, 6, 9, 43, 413000, tzinfo=tzlocal()),
  'ETag': '"fdb3803425be82d35c6ed178a14c4305"',
  'Size': 146,
  'StorageClass': 'STANDARD',
  'Owner': {'DisplayName': '52aa9e0c-0aa7-45e5-b231-1a65d1a57a1a',
   'ID': '52aa9e0c-0aa7-45e5-b231-1a65d1a57a1a'}},
 {'Key': 'Cond4n_e104.parquet',
  'LastModified': datetime.datetime(2021, 9, 30, 8, 7, 8, 655000, tzinfo=tzlocal()),
  'ETag': '"09882233f58af6073d0516933749873c"',
  'Size': 4208,
  'StorageClass': 'STANDARD',
  'Owner': {'DisplayName': '52aa9e0c-0aa7-45e5-b231-1a65d1a57a1a',
   'ID': '52aa9e0c-0aa7-45e5-b231-1a65d1a57a1a'}},
 {'Key': 'Cond4new.cs